In [1]:
import tensorflow as tf

In [328]:
from sklearn.model_selection import train_test_split
import time
import warnings
warnings.filterwarnings('ignore')

# 1. 랜덤 데이터 생성
- users + items + gender + click(label)

In [270]:
sampleSize = 30000
NumberOfUser = 1000
NumberOfItem = 100

# random data 생성
np.random.seed(0)
users_ls = np.random.randint(NumberOfUser,size=sampleSize)
users_ls_str = list(map(lambda x : str(x),users_ls))
items_ls = np.random.randint(NumberOfItem,size=sampleSize)
items_ls_str = list(map(lambda x : str(x),items_ls))
gender_ls = np.random.randint(2,size=sampleSize)
gender_ls_str = list(map(lambda x : str(x),gender_ls))
click_ls = np.random.randint(2,size=sampleSize)
Xdata2 = [{'user': users_ls_str[i], 'items': items_ls_str[i]} \
         for i in range(sampleSize)]
Xdata3 = [{'user': users_ls_str[i], 'items': items_ls_str[i], 'gender': gender_ls_str[i]} \
         for i in range(sampleSize)]

df = pd.DataFrame(Xdata3)
df['click'] = click_ls
df.head()
# users_onehot = tf.reshape(tf.one_hot(users,depth=NumberOfUser),shape=(sampleSize,NumberOfUser))
# items_onehot = tf.reshape(tf.one_hot(items,depth=NumberOfItem),shape=(sampleSize,NumberOfItem))
# features = tf.concat((users_onehot, items_onehot, gender, click), axis=1)
# print("features : 처음 {}칸 user 정보,그 뒤로 {}칸 item, 그 뒤로 1칸은 gender, 마지막 1칸은 click or not(label)".format(NumberOfUser,NumberOfItem))
# features

,gender,items,user,click
0,1,26,684,0
1,1,56,559,0
2,0,83,629,0
3,0,3,192,0
4,1,18,835,1


In [271]:
v2 = DictVectorizer()
X_train2 = v2.fit_transform(Xdata2)
#X_test = v.transform(test_data)
(X_train2.toarray()[0]).sum()

2.0

In [280]:
Y_train = click_ls
Y_train[Y_train==0]=-1  # 0 -> -1
# Y_train

In [281]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_train2, Y_train, test_size=0.3) # for regression

## (1) pylibfm

In [302]:
# Build and train a Factorization Machine
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task='classification', initial_learning_rate=0.001, learning_rate_schedule="optimal")
fm.fit(Xtrain,Ytrain)

# Evaluate
preds = fm.predict(Xtest)


Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training log loss: 0.69340
-- Epoch 2
Training log loss: 0.69281
-- Epoch 3
Training log loss: 0.69238
-- Epoch 4
Training log loss: 0.69190
-- Epoch 5
Training log loss: 0.69145
-- Epoch 6
Training log loss: 0.69100
-- Epoch 7
Training log loss: 0.69060
-- Epoch 8
Training log loss: 0.69020
-- Epoch 9
Training log loss: 0.68973
-- Epoch 10
Training log loss: 0.68943
-- Epoch 11
Training log loss: 0.68907
-- Epoch 12
Training log loss: 0.68868
-- Epoch 13
Training log loss: 0.68830
-- Epoch 14
Training log loss: 0.68804
-- Epoch 15
Training log loss: 0.68772
-- Epoch 16
Training log loss: 0.68738
-- Epoch 17
Training log loss: 0.68705
-- Epoch 18
Training log loss: 0.68677
-- Epoch 19
Training log loss: 0.68648
-- Epoch 20
Training log loss: 0.68619
-- Epoch 21
Training log loss: 0.68589
-- Epoch 22
Training log loss: 0.68560
-- Epoch 23
Training log loss: 0.68533
-- Epoch 24
Training log loss: 0.685

In [311]:
preds[preds < 0.5] = -1
preds[preds >= 0.5] = 1

In [314]:
from sklearn.metrics import mean_squared_error
print("FM MSE:", mean_squared_error(Ytest,preds))
print('acc:', accuracy_score(Ytest, preds))

FM MSE: 2.017777777777778
acc: 0.4955555555555556


## (2) Fast FM

In [329]:
from fastFM import sgd
from sklearn.metrics import accuracy_score, roc_auc_score

# check time
start_time = time.time()

fm = sgd.FMClassification(n_iter=10000, init_stdev=0.2, rank=4, step_size=0.2)
fm.fit(Xtrain, Ytrain)
Ypred = fm.predict(Xtest)
# probability 형태로도 예측 가능
Ypred_proba = fm.predict_proba(Xtest)
#print(y_pred, y_pred_proba)
print('acc:', accuracy_score(Ytest, Ypred))
print('auc:', roc_auc_score(Ytest, Ypred_proba))

# check time
print("--- %s seconds ---" % (time.time() - start_time))

acc: 0.499
auc: 0.49672211309704883
--- 0.017866849899291992 seconds ---


## (3) SVM polynomial kernel

In [324]:
from sklearn.svm import SVC

In [330]:
# check time
start_time = time.time()

clf = SVC(C=1.0, kernel='poly', degree=2, gamma = 'auto')
classifier = clf.fit(Xtrain,Ytrain)

YpredSVC = classifier.predict(Xtest)
#print(y_pred, y_pred_proba)
print('acc:', accuracy_score(Ytest, YpredSVC))

# check time
print("--- %s seconds ---" % (time.time() - start_time))

acc: 0.4991111111111111
--- 11.753458261489868 seconds ---


- fastFM은 0.0178초 걸림.
- SVC는 11.753초가 걸림.
- accuracy는 동일하지만 fast FM이 압도적으로 빠름.

# 2. Fast FM data set -> FM

In [315]:
from fastFM.datasets import make_user_item_regression

# This sets up a small test dataset.
X, y, _ = make_user_item_regression(n_user=100, n_item=100)
#X_train, X_test, y_train, y_test = train_test_split(X, y) # for regression

In [320]:
X.shape, y.shape

((10000, 200), (10000,))

In [321]:
# Convert dataset to binary classification task.
y_labels = np.ones_like(y)
y_labels[y < np.mean(y)] = -1
#print(y_labels)
X_train, X_test, y_train, y_test = train_test_split(X, y_labels)
#print(X_train.toarray())

- n_iter : The number of iterations
- init_stdev : the standard deviation used to initialize the model parameter and the number of hidden variables rank per feature. 
- This are the parameters that have to be specified for every solver and task. 
- The ALS solver requires in addition the regularization values for the first l2_reg_w and second order l2_reg_V interactions.

In [322]:
from fastFM import sgd
fm = sgd.FMClassification(n_iter=1000, init_stdev=0.1, l2_reg_w=0,
                          l2_reg_V=0, rank=2, step_size=0.1)
fm.fit(X_train, y_train)
y_pred = fm.predict(X_test)
#y_pred

# probability 형태로도 예측 가능
y_pred_proba = fm.predict_proba(X_test)
#y_pred_proba

In [323]:
from sklearn.metrics import accuracy_score, roc_auc_score
print('acc:', accuracy_score(y_test, y_pred))
print('auc:', roc_auc_score(y_test, y_pred_proba))

acc: 0.8336
auc: 0.9223819851284363


----
----

# pyFM (데이터 로드하는것 굳)

In [136]:
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

In [331]:
# Read in data
def loadData(filename,path="ml-100k/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (user,movieid,rating,ts)=line.split('\t')
            data.append({ "user_id": str(user), "movie_id": str(movieid)})
            y.append(float(rating))
            users.add(user)
            items.add(movieid)

    return (data, np.array(y), users, items)

(train_data, y_train, train_users, train_items) = loadData("ua.base")
(test_data, y_test, test_users, test_items) = loadData("ua.test")

In [334]:
train_data[0], y_train[0]

({'user_id': '1', 'movie_id': '1'}, 5.0)

In [146]:
v = DictVectorizer()
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)

In [153]:
X_train.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [155]:
# Build and train a Factorization Machine
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

fm.fit(X_train,y_train)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.50205
-- Epoch 2
Training MSE: 0.48544
-- Epoch 3
Training MSE: 0.47044
-- Epoch 4
Training MSE: 0.45698
-- Epoch 5
Training MSE: 0.44480
-- Epoch 6
Training MSE: 0.43381
-- Epoch 7
Training MSE: 0.42378
-- Epoch 8
Training MSE: 0.41445
-- Epoch 9
Training MSE: 0.40587
-- Epoch 10
Training MSE: 0.39774
-- Epoch 11
Training MSE: 0.39014
-- Epoch 12
Training MSE: 0.38299
-- Epoch 13
Training MSE: 0.37629
-- Epoch 14
Training MSE: 0.36992
-- Epoch 15
Training MSE: 0.36386
-- Epoch 16
Training MSE: 0.35797
-- Epoch 17
Training MSE: 0.35237
-- Epoch 18
Training MSE: 0.34699
-- Epoch 19
Training MSE: 0.34178
-- Epoch 20
Training MSE: 0.33680
-- Epoch 21
Training MSE: 0.33196
-- Epoch 22
Training MSE: 0.32727
-- Epoch 23
Training MSE: 0.32272
-- Epoch 24
Training MSE: 0.31831
-- Epoch 25
Training MSE: 0.31403
-- Epoch 26
Training MSE: 0.30985
-- Epoch 27
Training MSE: 0.30583
-- Epoch 28
Tra

In [156]:
# Evaluate
preds = fm.predict(X_test)
from sklearn.metrics import mean_squared_error
print("FM MSE: %.4f" % mean_squared_error(y_test,preds))

FM MSE: 0.7528
